In [1]:
import os

In [2]:
%pwd

'c:\\Users\\shash\\fetch_assessment\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\shash\\fetch_assessment'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class SplitingDataConfig:
    root_dir: Path
    data_path: Path
    look_back: int
    split:float

In [6]:
from fetch_assessment.constants import *
from fetch_assessment.utils.common import read_yaml, create_directories

Importing constants
Config path: config\config.yaml
Params path: params.yaml


In [15]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_Spliting_Data_config(self) -> SplitingDataConfig:
        config = self.config.spliting_data
        params = self.params.splitingparameters
        create_directories([config.root_dir])

        spliting_data_config = SplitingDataConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            look_back=params.look_back,
            split=params.split
        )

        return spliting_data_config

In [16]:
import os
import numpy as np
import pandas as pd
from fetch_assessment.logging import logger
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [21]:
class Spliting_Data:
    def __init__(self,config:SplitingDataConfig):
        self.config = config

    
    def create_dataset(self,data,look_back):
        X, Y = [], []
        for i in range(len(data) - look_back):
            X.append(data[i:(i + look_back)])
            Y.append(data[i + look_back])
        return np.array(X), np.array(Y)

    def spliting_data(self,X,y):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        scaler = MinMaxScaler(feature_range=(0, 1))
        X_train_scaled = np.array([scaler.fit_transform(x) for x in X_train])
        X_test_scaled = np.array([scaler.transform(x) for x in X_test])
        scaler_y = MinMaxScaler(feature_range=(0, 1))
        y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1))
        y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1))        

        return X_train_scaled,X_test_scaled,y_train_scaled,y_test_scaled

    def saving_data(self):
        X,y=self.create_dataset(pd.read_csv(self.config.data_path).drop(["# Date"],axis=1).values,self.config.look_back)
        X_train, X_test, y_train, y_test=self.spliting_data(X,y) 
        np.save(os.path.join(self.config.root_dir,"X_train.npx"),X_train)
        np.save(os.path.join(self.config.root_dir,"X_test.npx"),X_test)
        np.save(os.path.join(self.config.root_dir,"y_train.npx"),y_train)
        np.save(os.path.join(self.config.root_dir,"y_test.npx"),y_test)

In [22]:
try:
    config = ConfigurationManager()
    Split_Data_config = config.get_Spliting_Data_config()
    Split_Data = Spliting_Data(config=Split_Data_config)
    Split_Data.saving_data()
except Exception as e:
    raise e

[2023-11-12 22:41:42,137: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-11-12 22:41:42,139: INFO: common: yaml file: params.yaml loaded successfully]
[2023-11-12 22:41:42,140: INFO: common: created directory at: artifacts]
[2023-11-12 22:41:42,142: INFO: common: created directory at: artifacts/data_splting_data]
